In [1]:
# hit API 
import requests

# parsing data
import pandas as pd
from datetime import date

# scraping progress
from tqdm import tqdm

# connecting to db
import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [2]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("DANMAR LINES")

In [3]:
# key to map key from json request to key used in db
Mapping = { "Estimated Vessel Departure"  : "ETD",
            "Actual Vessel Departure" : "ATD",
            "Estimated Vessel Arrival" : "ETA",
            "Actual Vessel Arrival" : "ATA"}

In [4]:
bl_list

[]

In [5]:
list_of_dict = []
sign = False

for i, bls in enumerate(tqdm(bl_list)):
    try:
    
        print("BL in Process: " ,bls)
        
        # hitting API
        url = "https://api.dhl.com/dgff/transportation/shipment-tracking"
        querystring = {"housebill": bls}
        headers = {
            "content-type": "application/json",
            "DHL-API-Key": "qEHPTpIAc13hHfjB6H6gJQ0vEfmP6GhA"
        }

        # pasting json response
        response = requests.request("GET", url, headers=headers, params=querystring)
        json_data = response.json() if response and response.status_code == 200 else None

        temp_dict={}
        
        # list of containers in 1 bl
        list_of_containers=[]
        for i in json_data['ShipmentTracking']['Shipment']['TransportUnits']['TransportUnit']:
            list_of_containers.append(i['TransportUnitID'])

        print(list_of_containers)

        # parsing every data per container
        for c, container in enumerate(tqdm(list_of_containers)):
            current_dict={}
            current_dict.update({"Liners" : "Danmar Lines / DHL"})
            current_dict.update({"BL Number" : bls})
            current_dict.update({"Container Number" : container})
            current_dict.update({"From" : json_data['ShipmentTracking']['Shipment']['Origin']['LocationName'][:json_data['ShipmentTracking']['Shipment']['Origin']['LocationName'].index(",")]})
            current_dict.update({"To" : json_data['ShipmentTracking']['Shipment']['Destination']['LocationName']})
            
            for j, status_dict in enumerate (json_data['ShipmentTracking']['Shipment']['Timestamps']['Timestamp']):
                status_name = status_dict.get('TimestampDescription')
                status_date = status_dict.get('TimestampDateTime')
                case_data = {status_name:status_date[:10]}
                temp_dict.update(case_data)
                
            for old_key, new_key in Mapping.items():
                try:
                    current_dict[new_key] = temp_dict[old_key]
                except Exception as e:
                    current_dict[new_key] = None
                    print("THIS IS ERROR====>",e)
                    list_of_dict.append(current_dict)
                    sign = True

            if sign == False:
                list_of_dict.append(current_dict)
            print(bls, container,"done")
    except Exception as e:
        print(bls, "THIS IS ERROR====>",e)

0it [00:00, ?it/s]


In [6]:
list_of_dict

[]

In [7]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict)

Today's Collection Name ===>  all_tracking_Jan-08-2024
Today's Collection Has Been Made


TypeError: documents must be a non-empty list

In [8]:
# exporting scraped data to excel
df = pd.DataFrame(list_of_dict)
excel_file_path = 'export excel/DHL.xlsx'
df.to_excel(excel_file_path, index=False)